In [1]:
import mlflow 
from mlflow.models import infer_signature
import mlflow.pyfunc

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
X, y = datasets.load_iris(return_X_y=True)
y = (y == 0).astype(int)  # Binary: Setosa vs Not Setosa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# normalization
def normalize(X):
    minx = np.min(X, axis=0)
    maxx = np.max(X, axis=0)
    return (X - minx) / (maxx - minx)

X_train = normalize(X_train)
X_test = normalize(X_test)

# Sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# logistic regression
def cost_func(y_true, y_pred):
    m = y_true.shape[0]
    cost = - (1 / m) * np.sum(
        y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15)
    )
    return cost

def logistic_regression(X, y, lr=0.01, epochs=1000):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0

    for epoch in range(epochs):
        # Compute predictions
        y_pred = sigmoid(np.dot(X, weights) + bias)
        
        # Compute gradients
        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)
        
        # Update parameters
        weights -= lr * dw
        bias -= lr * db
        

        # Calculate cost to monitor learning progress
        cost_history = []
        cost = cost_func(y, y_pred)
        cost_history.append(cost)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Cost: {cost:.4f}")

    return weights, bias



# Prediction
def predict(X, weights, bias):
    return sigmoid(np.dot(X, weights) + bias)

# Train and predict
weights, bias = logistic_regression(X_train, y_train)
y_pred_prob = predict(X_test, weights, bias)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Epoch 0, Cost: 0.6931
Epoch 100, Cost: 0.6016
Epoch 200, Cost: 0.5499
Epoch 300, Cost: 0.5145
Epoch 400, Cost: 0.4867
Epoch 500, Cost: 0.4629
Epoch 600, Cost: 0.4417
Epoch 700, Cost: 0.4224
Epoch 800, Cost: 0.4046
Epoch 900, Cost: 0.3881
Accuracy: 1.0


In [4]:
model_params = {
    "weights": weights,
    "bias": bias,
    "learning_rate": 0.1,
    "epochs": 1000
}
print(model_params)


{'weights': array([-0.5766677 ,  0.46959185, -1.01091752, -1.03441357]), 'bias': np.float64(0.06359140662395034), 'learning_rate': 0.1, 'epochs': 1000}


In [5]:
def confusion_matrix(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))  # True Positive
    FP = np.sum((y_true == 0) & (y_pred == 1))  # False Positive
    TN = np.sum((y_true == 0) & (y_pred == 0))  # True Negative
    FN = np.sum((y_true == 1) & (y_pred == 0))  # False Negative

    return TP, FP, TN, FN

TP, FP, TN, FN = confusion_matrix(y_test, y_pred)

print(f"Confusion Matrix:")
print(f"TP: {TP}, FP: {FP}")
print(f"TN: {TN}, FN: {FN}")


Confusion Matrix:
TP: 8, FP: 0
TN: 22, FN: 0


In [6]:
def sensitivity(TP, FN):
    return TP / (TP + FN) if (TP + FN) != 0 else 0

def specificity(TN, FP):
    return TN / (TN + FP) if (TN + FP) != 0 else 0

def precision(TP, FP):
    return TP / (TP + FP) if (TP + FP) != 0 else 0

def F_score(prec, recall):
    return 2*(prec*recall)/(prec + recall) if prec + recall != 0 else 0

recall = sensitivity(TP,FN)
spec = specificity(TN,FP)
prec = precision(TP,FP)
Fscore = F_score(prec,recall)

print(recall,spec,prec,Fscore)



1.0 1.0 1.0 1.0


In [ ]:
import pickle
import os
# Save weights and bias
os.makedirs("model_artifacts", exist_ok=True)
with open("model_artifacts/weights.pkl", "wb") as f:
    pickle.dump(weights, f)
with open("model_artifacts/bias.pkl", "wb") as f:
    pickle.dump(bias, f)
    
class CustomLogisticRegression(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.weights = pickle.load(open(context.artifacts["weights"], "rb"))
        self.bias = pickle.load(open(context.artifacts["bias"], "rb"))

    def predict(self, context, model_input):
        return (1 / (1 + np.exp(-(model_input @ self.weights + self.bias))) >= 0.5).astype(int)


In [15]:
mlflow.set_experiment("Logistic Regression")

with mlflow.start_run():
    mlflow.log_params({
        "weights":weights,
        "Bias":bias,
        "learning_rate": 0.01,
        "epochs": 1000,
        "model_type": "LogisticRegression"
    })
    
    mlflow.log_metrics({
        "accuracy": accuracy,
        "sensitivity": recall,
        "specificity": spec,
        "precision":prec,
        "F-measure": Fscore
    })
    
    mlflow.log_artifact("model_artifacts/weights.pkl", artifact_path="weights")
    mlflow.log_artifact("model_artifacts/bias.pkl", artifact_path="bias")

    mlflow.pyfunc.log_model(
        artifact_path="logreg_model",
        signature = infer_signature(X_train, y_pred),

        python_model=CustomLogisticRegression(),
        registered_model_name= "logistic regression",
        artifacts={
            "weights": "model_artifacts/weights.pkl",
            "bias": "model_artifacts/bias.pkl"
        }
    )
    

Registered model 'logistic regression' already exists. Creating a new version of this model...
2025/04/12 18:27:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic regression, version 2


🏃 View run upset-stag-684 at: http://127.0.0.1:5000/#/experiments/384315332130507177/runs/41db3d9d02374d6697bdf5cfbe192aca
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/384315332130507177


Created version '2' of model 'logistic regression'.
